In [ ]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import pickle
from sklearn.metrics import confusion_matrix
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from matplotlib import pyplot
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc,recall_score,precision_score
from sklearn.metrics import accuracy_score, make_scorer, recall_score, precision_score, f1_score
from sklearn.ensemble import VotingClassifier
import sys

# **Reading Dataset**

In [ ]:
dataset = pd.read_csv('https://raw.githubusercontent.com/jusung1459/data/main/data.txt')
datasetTest = pd.read_csv('https://raw.githubusercontent.com/jusung1459/data/main/locationMergedTest.csv')
print(dataset.describe())
dataset['date_confirmation'] = pd.to_datetime(dataset['date_confirmation'])

                 age     Confirmed  ...  Incidence_Rate  Case-Fatality_Ratio
count  367636.000000  3.676360e+05  ...   367636.000000        367636.000000
mean       43.291751  4.471816e+05  ...      948.291557             2.873130
std        12.968343  3.990631e+05  ...      833.222440             1.822906
min         0.000000  1.000000e+00  ...        0.029070             0.000000
25%        43.000000  1.112900e+05  ...      360.749785             1.939005
50%        43.000000  3.491670e+05  ...      948.072083             2.723007
75%        43.000000  5.309080e+05  ...      948.291557             2.873130
max       106.000000  1.167496e+06  ...     6072.296061            18.494473

[8 rows x 7 columns]


# **Preprocessing**

In [ ]:
train, validation = train_test_split(dataset, test_size = 0.20, random_state = 42)

In [ ]:
x_train = train[['age', 'sex', 'date_confirmation', 'combinedKey', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Incidence_Rate', 'Case-Fatality_Ratio']]
y_train = train[['outcome']]

x_validation = validation[['age', 'sex', 'date_confirmation', 'combinedKey', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Incidence_Rate', 'Case-Fatality_Ratio']]
y_validation = validation[['outcome']]

In [ ]:
# encoder (categorical to numerical)
ord_enc = OrdinalEncoder()
encoded_dataSet = dataset.copy()
encoded_dataSet["sex"] = ord_enc.fit_transform(encoded_dataSet[["sex"]])
encoded_dataSet["outcome"] = ord_enc.fit_transform(encoded_dataSet[["outcome"]])
encoded_dataSet["combinedKey"] = ord_enc.fit_transform(encoded_dataSet[["combinedKey"]])
encoded_dataSet["date_confirmation"] = ord_enc.fit_transform(encoded_dataSet[["date_confirmation"]])
train_encoded, validation_encoded = train_test_split(encoded_dataSet, test_size = 0.20, random_state = 42)
x_train_encoded = train_encoded[['age', 'sex', 'date_confirmation', 'combinedKey', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Incidence_Rate', 'Case-Fatality_Ratio']]
y_train_encoded = train_encoded[['outcome']]

x_validation_encoded = validation_encoded[['age', 'sex', 'date_confirmation', 'combinedKey', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Incidence_Rate', 'Case-Fatality_Ratio']]
y_validation_encoded = validation_encoded[['outcome']]

encoded_dataSet_X = encoded_dataSet[['age', 'sex', 'date_confirmation', 'combinedKey', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Incidence_Rate', 'Case-Fatality_Ratio']]
encoded_dataSet_Y = encoded_dataSet[['outcome']]

In [ ]:
# encoder (categorical to numerical) for datasetTest data
ord_enc = OrdinalEncoder()
encoded_dataSetTest = datasetTest.copy()
encoded_dataSetTest["date_confirmation"] = encoded_dataSetTest["date_confirmation"].fillna('2020-11-05')
encoded_dataSetTest["Confirmed"] = encoded_dataSetTest["Confirmed"].fillna(0)
encoded_dataSetTest["Deaths"] = encoded_dataSetTest["Deaths"].fillna(0)
encoded_dataSetTest["Recovered"] = encoded_dataSetTest["Recovered"].fillna(0)
encoded_dataSetTest["Active"] = encoded_dataSetTest["Active"].fillna(0)
encoded_dataSetTest["Incidence_Rate"] = encoded_dataSetTest["Incidence_Rate"].fillna(0)
encoded_dataSetTest["Case-Fatality_Ratio"] = encoded_dataSetTest["Case-Fatality_Ratio"].fillna(0)

encoded_dataSetTest["sex"] = ord_enc.fit_transform(encoded_dataSetTest[["sex"]])
encoded_dataSetTest["combinedKey"] = ord_enc.fit_transform(encoded_dataSetTest[["combinedKey"]])
encoded_dataSetTest["date_confirmation"] = ord_enc.fit_transform(encoded_dataSetTest[["date_confirmation"]])
x_encoded_dataSetTest = encoded_dataSetTest[['age', 'sex', 'date_confirmation', 'combinedKey', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Incidence_Rate', 'Case-Fatality_Ratio']]

# **Building & Tuning Decision Tree**

In [ ]:
dtModel = DecisionTreeClassifier(random_state = 10)
parameters = {
    'max_depth': [10,20,30], #determines how deep the tree is grown
    'criterion': ["gini", "entropy"], #criteria to measure quality of split
    'splitter': ["best"] #because best always produces greater accuracies for larger datasets
}

scorers = {
    'f1_weighted' : make_scorer(f1_score, average='weighted'),
    'precision_weighted': make_scorer(precision_score, average='weighted'),
    'recall_weighted': make_scorer(recall_score , average='weighted'),
    'accuracy': make_scorer(accuracy_score),
    'recall[deceased]' : make_scorer(recall_score,average=None,labels=[0])
}

cv = StratifiedKFold(n_splits=5, shuffle = True)

dtClassification = GridSearchCV(dtModel, 
                    param_grid = parameters,
                    scoring = scorers, 
                    n_jobs = -1, 
                    cv = cv.split(x_train_encoded, y_train_encoded),
                    refit = 'recall[deceased]')

dtClassification.fit(x_train_encoded, y_train_encoded)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x7f971e11e850>,
             error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprec...
             pre_dispatch='2*n_jobs', refit='recall[deceased]',
             return_train_score=False,
             scoring={'accuracy': make_scorer(accuracy_score),
                      '

In [ ]:
decisiontree = pd.DataFrame(dtClassification.cv_results_)
decisiontree = decisiontree[['param_max_depth',	'param_criterion'	, 'param_splitter' , 'mean_test_f1_weighted', 'mean_test_precision_weighted', 'mean_test_recall_weighted','mean_test_recall[deceased]', 'mean_test_accuracy', 'rank_test_accuracy']]
decisiontree = decisiontree.sort_values(by='mean_test_recall[deceased]')
decisiontree.to_csv('xgb.csv',index=False)

In [ ]:
print(dtClassification.best_params_)
print(dtClassification.best_score_)

{'criterion': 'gini', 'max_depth': 30, 'splitter': 'best'}
0.07825526428878385


In [ ]:
decisiontree

,param_max_depth,param_criterion,param_splitter,mean_test_f1_weighted,mean_test_precision_weighted,mean_test_recall_weighted,mean_test_recall[deceased],mean_test_accuracy,rank_test_accuracy
0,10,gini,best,0.802168,0.828323,0.819876,0.043040,0.819876,6
3,10,entropy,best,0.806239,0.821976,0.820345,0.046958,0.820345,5
4,20,entropy,best,0.824614,0.829806,0.833024,0.075460,0.833024,1
1,20,gini,best,0.824546,0.829387,0.832721,0.076857,0.832721,4
5,30,entropy,best,0.824409,0.829508,0.832803,0.077417,0.832803,3
2,30,gini,best,0.824564,0.829701,0.832891,0.078255,0.832891,2


In [ ]:
yPred = dtClassification.predict(x_validation_encoded)
print("Accuracy Test {0:.2f}%".format(100*accuracy_score(y_validation_encoded, yPred)))
print(classification_report(y_validation_encoded, yPred))

Accuracy Test 83.34%
              precision    recall  f1-score   support

         0.0       0.32      0.09      0.14       921
         1.0       0.72      0.88      0.79     24912
         2.0       0.99      0.99      0.99     30119
         3.0       0.74      0.54      0.62     17576

    accuracy                           0.83     73528
   macro avg       0.69      0.62      0.64     73528
weighted avg       0.83      0.83      0.82     73528



In [ ]:
# dtModel2 = DecisionTreeClassifier(random_state = 10, max_depth = 30, criterion = 'gini', splitter = 'best')
# dtModel2.fit(x_train_encoded, y_train_encoded)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=30, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=10, splitter='best')

In [ ]:
# yPred = dtModel2.predict(x_validation_encoded)
# print("Accuracy Test {0:.2f}%".format(100*accuracy_score(y_validation_encoded, yPred)))
# print(classification_report(y_validation_encoded, yPred))

Accuracy Test 83.34%
              precision    recall  f1-score   support

         0.0       0.32      0.09      0.14       921
         1.0       0.72      0.88      0.79     24912
         2.0       0.99      0.99      0.99     30119
         3.0       0.74      0.54      0.62     17576

    accuracy                           0.83     73528
   macro avg       0.69      0.62      0.64     73528
weighted avg       0.83      0.83      0.82     73528



In [ ]:
print(confusion_matrix(y_validation_encoded, yPred))

[[   80   581    44   216]
 [   39 21922     7  2944]
 [   78    19 29806   216]
 [   51  7830   223  9472]]


# **Building & Tuning KNN**

In [ ]:
knn_model = KNeighborsClassifier()

param_dist = {  
            'n_neighbors': [3,5,10,50,100],
            'weights': ['uniform', 'distance'],
            'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']      
        }

scorers = {
    'f1_weighted' : make_scorer(f1_score, average='weighted'),
    'precision_weighted': make_scorer(precision_score, average='weighted'),
    'recall_weighted': make_scorer(recall_score , average='weighted'),
    'accuracy': make_scorer(accuracy_score),
    'recall[deceased]' : make_scorer(recall_score,average=None,labels=[0])
}


# scorers = ['f1_weighted','precision_weighted','recall_weighted','accuracy']


skf = StratifiedKFold(n_splits=5, shuffle = True)

clf_knn = RandomizedSearchCV(knn_model, 
                    param_dist, 
                    scoring = scorers, 
                    n_jobs = -1, 
                    cv = skf.split(x_train_encoded, y_train_encoded),
                    n_iter = 30,
                    refit = 'recall[deceased]')

clf_knn.fit(x_train_encoded, y_train_encoded)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [ ]:
knn = pd.DataFrame(clf_knn.cv_results_)
knn = knn[['param_n_neighbors','param_algorithm',	'param_weights'	, 'mean_test_f1_weighted', 'mean_test_precision_weighted', 'mean_test_recall_weighted','mean_test_recall[deceased]', 'mean_test_accuracy', 'rank_test_accuracy']]
knn = knn.sort_values(by='rank_test_accuracy')
knn.to_csv('knn.csv',index=False)
knn

In [ ]:
y_test_pred = clf_knn.predict(x_validation_encoded)
print(accuracy_score(y_validation_encoded,y_test_pred) )
print("Accuracy Test {0:.2f}%".format(100*accuracy_score(y_validation_encoded,y_test_pred)))
print(classification_report(y_validation_encoded, y_test_pred))

In [ ]:
print(clf_knn.best_params_)
print(clf_knn.best_score_)

In [ ]:
print(confusion_matrix(y_validation_encoded, y_test_pred))

# **Building & Tuning XGB**


In [ ]:
# tuning parameters
n_estimators = [2500, 2800, 3000]
learning_rate = [0.01,0.001, 0.015]
max_depth=[12] 
gamma=[.1,.2,.3,.4,.5]

# n_estimators = [800, 1000, 1500]
# learning_rate = [0.01, 0.1, 1]
# max_depth=[10, 12, 15] 
# gamma=[.5, .1, .7]

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42, eval_metric="merror", use_label_encoder=False)

for n in n_estimators:
    for l in learning_rate:
        for m in max_depth:
            for g in gamma:
                n_estimators = [n]
                learning_rate = [l]
                max_depth=[m] 
                gamma=[g]
                scoring = ['recall_micro', 'accuracy']
                param_grid = dict(learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth, gamma=gamma)
                grid_search = GridSearchCV(xgb_model, param_grid, scoring=scoring, refit='accuracy')
                grid_result = grid_search.fit(x_train_encoded, y_train_encoded)
                
                print("n:" + str(n) + " l:"+str(l) + " m:"+str(m) + " g:"+str(g))
                print(classification_report(grid_result.best_estimator_.predict(x_train_encoded), y_train_encoded))

In [ ]:
 # best model parameters n:2500 l:0.01 m:12 g:0.3
xgb_model = xgb.XGBClassifier(use_label_encoder=False, objective="multi:softprob", random_state=42, eval_metric="merror", n_estimators=2500, learning_rate=0.01, max_depth=12, gamma=0.3)
xgb_model.fit(x_train_encoded, y_train_encoded)

In [ ]:
# Classification Accuracy
yVald_pred_xgb = xgb_model.predict(x_validation_encoded)
yTrain_pred_xgb = xgb_model.predict(x_train_encoded)

print(accuracy_score(y_validation_encoded, yVald_pred_xgb))
print(confusion_matrix(y_validation_encoded, yVald_pred_xgb))

print(accuracy_score(y_train_encoded, yTrain_pred_xgb))
print(confusion_matrix(y_train_encoded, yTrain_pred_xgb))

# Predicting CasesTest.csv outcomes

In [ ]:
 # best model parameters n:2500 l:0.01 m:12 g:0.3
xgb_model = xgb.XGBClassifier(use_label_encoder=False, objective="multi:softprob", random_state=42, eval_metric="merror", n_estimators=2500, learning_rate=0.01, max_depth=12, gamma=0.3)
xgb_model.fit(x_train_encoded, y_train_encoded)

# best model for decision tree Classifier (gini, 20, best)
dtModel = DecisionTreeClassifier(criterion='gini', max_depth=30, splitter='best')
dtModel.fit(x_train_encoded, y_train_encoded)

# best model for KNN classifier (uniform, 3, brute)
knnModel = KNeighborsClassifier(weights='uniform', n_neighbors=3, algorithm='brute')
knnModel.fit(x_train_encoded, y_train_encoded)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyboardInterrupt: ignored

In [ ]:
eclf1 = VotingClassifier(estimators=[('xgb', xgb_model), ('dt', dtModel), ('knn', knnModel)], voting='soft')
eclf1 = eclf1.fit(encoded_dataSet_X, encoded_dataSet_Y) 

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
ensembls = eclf1.predict(x_encoded_dataSetTest)

In [ ]:
print(ensembls.shape)
predictions = []
for i in ensembls:
    if i == 0.0:
        predictions.append('deceased')
    if i == 1.0:
        predictions.append('hospitalized')
    if i == 2.0:
        predictions.append('nonhospitalized')
    if i == 3.0:
        predictions.append('recovered')
        
count = 1
with open('predictions.txt', 'w') as f:
    for row in predictions:
        if count == 46500:
            f.write(row)
        else:
            f.write(row + '\n')
        count += 1

(46500,)